We will work today with 'momentum_data.xlsx' file. It contains several tabs:

- The 1st tab contains the momentum factor as an excess return: $\tilde{r}^{\text{mom}}$.

- The 3rd tab contains returns on portfolios corresponding to scored momentum deciles.

    - $r^{\text{mom(1)}}$ denotes the portfolio of stocks in the lowest momentum decile, the 'losers' with the lowest past returns.
    
    - $r^{\text{mom(10)}}$ denotes the portfolio of stocks in the highest momentum decile.
    
- The 4th tab gives portfolios sorted by momentum and size.

    - $r^{\text{momSU}}$ denotes the portfolio of small stocks in the top 3 deciles of momentum scores.
    
    - $r^{\text{momBD}}$ denotes the portfolio of big-stocks in the bottom 3 deciles of momentum scores.

### Task 1.

Check that momentum return, $\tilde{r}^{\text{mom:FF}}$, given in the second tab, is constructed:

$\tilde{r}^{\text{mom:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2-
(r^{\text{momBD}}+r^{\text{momSd}})/2$

In [20]:
import pandas as pd 


factors = pd.read_excel("momentum_data.xlsx", "factors (excess returns)", index_col=0)
momentum = pd.read_excel("momentum_data.xlsx", "momentum (excess returns)", index_col=0)
size_sorts = pd.read_excel("momentum_data.xlsx", "size_sorts (total returns)", index_col=0)
momentum["UMD"] == (size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - (size_sorts["BIG LoPRIOR"] + size_sorts["SMALL LoPRIOR"])/2
r_mom_FF = pd.DataFrame(index=momentum.index)
r_mom_FF["UMD 1"] = round(momentum["UMD"], 1)
r_mom_FF["UMD 2"] = round((size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - (size_sorts["BIG LoPRIOR"] + size_sorts["SMALL LoPRIOR"])/2, 1)
all(r_mom_FF["UMD 1"] == r_mom_FF["UMD 2"])

True

### Task 2

Fill in next table 1 with the appropriate stats for $\tilde{r}^{\text{mom:FF}}$. Please note, that $\tilde{r}^m$ is the excess market return, MKT; $\tilde{r}^v$ is the value portfolio, HML.

<img src='table1.png'>

In [21]:
indexes = ["1927-2022", "1927-1993", "1994-2008", "2009-2022"]
momentum_per = pd.DataFrame()
for df in indexes:
    momentum_per.loc[df, "mean"] = r_mom_FF.loc[df[:4]:df[-4:], "UMD 1"].mean()
    momentum_per.loc[df, "vol"] = r_mom_FF.loc[df[:4]:df[-4:], "UMD 1"].std()
    momentum_per.loc[df, "Sharpe"] = momentum_per.loc[df, "mean"] / momentum_per.loc[df, "vol"]
    momentum_per.loc[df, "skewness"] = r_mom_FF.loc[df[:4]:df[-4:], "UMD 1"].skew()
    momentum_per.loc[df, "corr_to_Rm"] = r_mom_FF.loc[df[:4]:df[-4:], "UMD 1"].corr(factors.loc[df[:4]:df[-4:], "MKT"])
    momentum_per.loc[df, "corr_to_Rv"] = r_mom_FF.loc[df[:4]:df[-4:], "UMD 1"].corr(factors.loc[df[:4]:df[-4:], "HML"])
    
momentum_per

,mean,vol,Sharpe,skewness,corr_to_Rm,corr_to_Rv
1927-2022,0.000609,0.049822,0.012228,-2.188826,-0.345801,-0.405482
1927-1993,0.000871,0.047991,0.018142,-3.025223,-0.371853,-0.489162
1994-2008,0.003889,0.060134,0.064671,-0.482249,-0.250653,-0.109297
2009-2022,-0.004242,0.046000,-0.092226,-1.684882,-0.347623,-0.399363


### Task 3

Construct your own long-only implementation:

$\tilde{r}^{\text{momU:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2 - r^f.$

Note that you have to subtract the risk-free rate (RF) to get the excess return of this portfolio.

Fill out Table 2 for the data in the period 1994-2022.

<img src='table2.png'>

In [22]:
risk_free_rate = pd.read_excel("momentum_data.xlsx", "risk-free rate", index_col=0)
momentum_per_long = pd.DataFrame().rename_axis("1994-2022", axis=1)
r_mom_u_ff = pd.DataFrame()
r_mom_u_ff["UMD 1"] = momentum["UMD"].copy()
r_mom_u_ff["R_momU_FF"] = (size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - risk_free_rate["RF"]


momentum_per_long["mean"], momentum_per_long["vol"], momentum_per_long["Sharpe"], momentum_per_long["Skewness"] = (r_mom_u_ff.loc["1994": "2022"].mean(),
                                                                                                               r_mom_u_ff.loc["1994": "2022"].std(),
                                                                                                               r_mom_u_ff.loc["1994": "2022"].mean() / r_mom_u_ff.loc["1994": "2022"].std(),
                                                                                                               r_mom_u_ff.loc["1994": "2022"].skew())

for column in r_mom_u_ff.columns:
    momentum_per_long.loc[column, "corr_to_Rm"] = r_mom_u_ff.loc["1994": "2022", column].corr(factors["MKT"])
    momentum_per_long.loc[column, "corr_to_Rv"] = r_mom_u_ff.loc["1994": "2022", column].corr(factors["HML"])

momentum_per_long = momentum_per_long.rename({"UMD 1": "Long and short, (R_mom_FF)", "R_momU_FF": "Long only (R_momU_FF)"})
momentum_per_long

1994-2022,mean,vol,Sharpe,Skewness,corr_to_Rm,corr_to_Rv
"Long and short, (R_mom_FF)",0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601
Long only (R_momU_FF),0.009399,0.051645,0.181984,-0.482872,0.903499,-0.154667


### Task 4

Assess how sensitive the threshold for the 'winners' and 'losers' is in the results. Specifically, we compare three constructions:

- long the top 1 decile and short the bottom 1 deciles:

$\tilde{r}^{\text{momD1}} = r^{\text{mom(10)}}-r^{\text{mom(1)}}$

• long the top 3 deciles and short the bottom 3 deciles:

$\tilde{r}^{\text{momD3}} = 
(r^{\text{mom(8)}}+r^{\text{mom(9)}}+r^{\text{mom(10)}})/3-
(r^{\text{mom(1)}}+r^{\text{mom(2)}}+r^{\text{mom(3)}})/3 = 
\frac 13 \sum^{10}_{k=8} r^{\text{mom(k)}} + 
\frac 13 \sum^{3}_{k=1} r^{\text{mom(k)}}$
 

• long the top 5 deciles and short the bottom 5 deciles:

$\tilde{r}^{\text{momD5}} = 
\frac 15 \sum^{10}_{k=6} r^{\text{mom(k)}} + 
\frac 15 \sum^{5}_{k=1} r^{\text{mom(k)}}$

Compare all three constructions by filling out the stats in the table 3 below for the period 1994-2022.

<img src='table3.png'>

In [23]:
momentum_constr = pd.DataFrame().rename_axis("1994-2022", axis=1)
deciles_total_returns = pd.read_excel("momentum_data.xlsx", "deciles (total returns)", index_col=0)
deciles_total_returns = deciles_total_returns.loc["1994":"2022"]
r_mom_d = pd.DataFrame()
i = deciles_total_returns.columns
r_mom_d["R_mom_D1"] = deciles_total_returns[i[-1]] - deciles_total_returns[i[0]]
r_mom_d["R_mom_D3"] = ((deciles_total_returns[i[-3]] + deciles_total_returns[i[-2]] + deciles_total_returns[i[-1]]) / 3 - (deciles_total_returns[i[0]] + deciles_total_returns[i[1]] + deciles_total_returns[i[2]]) / 3)
r_mom_d["R_mom_D5"] = ((deciles_total_returns[i[-5]] + deciles_total_returns[i[-4]] + deciles_total_returns[i[-3]] + deciles_total_returns[i[-2]] + deciles_total_returns[i[-1]]) / 5 -
                       (deciles_total_returns[i[0]] + deciles_total_returns[i[1]] + deciles_total_returns[i[2]] + deciles_total_returns[i[3]] + deciles_total_returns[i[4]]) / 5)
momentum_constr["mean"], momentum_constr["vol"], momentum_constr["Sharpe"], momentum_constr["skewness"] = r_mom_d.mean(), r_mom_d.std(), r_mom_d.mean() / r_mom_d.std(), r_mom_d.skew()
for df in r_mom_d.columns:
    momentum_constr.loc[df, "corr_to_Rm"] = r_mom_d[df].corr(factors["MKT"])
    momentum_constr.loc[df, "corr_to_Rv"] = r_mom_d[df].corr(factors["HML"])
momentum_constr

1994-2022,mean,vol,Sharpe,skewness,corr_to_Rm,corr_to_Rv
R_mom_D1,0.006617,0.086011,0.076936,-1.323613,-0.335574,-0.236392
R_mom_D3,0.002850,0.055590,0.051273,-1.355380,-0.365386,-0.229882
R_mom_D5,0.001503,0.038690,0.038837,-1.435409,-0.357936,-0.227501


### Task 5
Use the data provided on both small-stock 'winners', $r^{\text{momSU}}$, and small-stock 'losers', $r^{\text{momSD}}$
to construct a small-stock momentum portfolio,

$
r_t^{\text{momS}} = r^{\text{momSU}} - r^{\text{momSD}}
$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$
r_t^{\text{momB}} = r^{\text{momBU}} - r^{\text{momBD}}
$

Fill out Table 4 over the sample 1994-2022.

<img src='table4.png'>

In [24]:
r_mom = pd.DataFrame()
r_mom["All stocks(R_mom_FF)"] = momentum["UMD"]
r_mom["Small stocks (R_t_mom_S)"] = size_sorts["SMALL HiPRIOR"] - size_sorts["SMALL LoPRIOR"]
r_mom["Large stocks (R_t_mom_B)"] = size_sorts["BIG HiPRIOR"] - size_sorts["BIG LoPRIOR"]
r_mom = r_mom.loc["1994":"2022"]

momentum_smlst_bgst = pd.DataFrame().rename_axis("1994-2022", axis=1)

momentum_smlst_bgst["mean"], momentum_smlst_bgst["vol"], momentum_smlst_bgst["Sharpe"], momentum_smlst_bgst["skewness"] = r_mom.mean(), r_mom.std(), r_mom.mean() / r_mom.std(), r_mom.skew()
for df in r_mom.columns:
    momentum_smlst_bgst.loc[df, "corr_to_Rm"] = r_mom[df].corr(factors["MKT"])
momentum_smlst_bgst

1994-2022,mean,vol,Sharpe,skewness,corr_to_Rm
All stocks(R_mom_FF),0.003946,0.048672,0.081075,-1.433554,-0.306139
Small stocks (R_t_mom_S),0.005228,0.049541,0.105531,-1.750109,-0.306908
Large stocks (R_t_mom_B),0.002660,0.052499,0.050667,-0.857615,-0.277984


### Task 6

Re-do Tables 2 and 4 but for the size portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- Size-sorted portfolios for value. "6 Portfolios Formed on Size and Book-to-Market (2 x 3)"

In [25]:
por_size_book_market = pd.read_csv("6_Portfolios_2x3.CSV", index_col=0, header=9, nrows=1155)
por_size_book_market.index = pd.to_datetime(por_size_book_market.index.astype("str"), format="%Y%m").to_period('M').to_timestamp('M')


r_mom_ff = pd.DataFrame()
r_mom_ff["R_mom_FF"] = ((por_size_book_market["BIG HiBM"] + por_size_book_market["SMALL HiBM"])/2 \
- (por_size_book_market["BIG LoBM"] + por_size_book_market["SMALL LoBM"])/2)
r_mom_ff.head()

,R_mom_FF
1926-07-31,-2.43355
1926-08-31,3.82270
1926-09-30,0.12640
1926-10-31,0.69920
1926-11-30,-0.51100


In [26]:
momentum_per_long = pd.DataFrame().rename_axis("1994-2022", axis=1)

r_mom_u_ff = r_mom_ff.copy()
r_mom_u_ff["R_momU_FF"] = (por_size_book_market["BIG HiBM"] + por_size_book_market["SMALL HiBM"])/2 - risk_free_rate["RF"]

momentum_per_long["mean"], momentum_per_long["vol"], momentum_per_long["Sharpe"], momentum_per_long["Skewness"] = (r_mom_u_ff.loc["1994": "2022"].mean(),
                                                                                                                   r_mom_u_ff.loc["1994": "2022"].std(),
                                                                                                                   r_mom_u_ff.loc["1994": "2022"].mean() / r_mom_u_ff.loc["1994": "2022"].std(),
                                                                                                                   r_mom_u_ff.loc["1994": "2022"].skew())

for column in r_mom_u_ff.columns:
    momentum_per_long.loc[column, "corr_to_Rm"] = r_mom_u_ff.loc["1994": "2022", column].corr(factors["MKT"])
    momentum_per_long.loc[column, "corr_to_Rv"] = r_mom_u_ff.loc["1994": "2022", column].corr(factors["HML"])

momentum_per_long = momentum_per_long.rename({"R_mom_FF": "Long and short, (R_mom_FF)", "R_momU_FF": "Long only (R_momU_FF)"})
momentum_per_long

1994-2022,mean,vol,Sharpe,Skewness,corr_to_Rm,corr_to_Rv
"Long and short, (R_mom_FF)",0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Long only (R_momU_FF),0.998077,5.495839,0.181606,-0.908702,0.866864,0.330352


In [27]:
r_mom = pd.DataFrame()
r_mom["All stocks(R_mom_FF)"] = r_mom_ff.copy()
r_mom["Small stocks (R_t_mom_S)"] = por_size_book_market["SMALL HiBM"] - por_size_book_market["SMALL LoBM"]
r_mom["Large stocks (R_t_mom_B)"] = por_size_book_market["BIG HiBM"] - por_size_book_market["BIG LoBM"]
r_mom = r_mom.loc["1994":"2022"]

momentum_smlst_bgst = pd.DataFrame().rename_axis("1994-2022", axis=1)

momentum_smlst_bgst["mean"], momentum_smlst_bgst["vol"], momentum_smlst_bgst["Sharpe"], momentum_smlst_bgst["skewness"] = r_mom.mean(), r_mom.std(), r_mom.mean() / r_mom.std(), r_mom.skew()
for df in r_mom.columns:
    momentum_smlst_bgst.loc[df, "corr_to_Rm"] = r_mom[df].corr(factors["MKT"])
momentum_smlst_bgst

1994-2022,mean,vol,Sharpe,skewness,corr_to_Rm
All stocks(R_mom_FF),0.132581,3.315845,0.039984,0.207751,-0.102250
Small stocks (R_t_mom_S),0.306017,3.750472,0.081594,0.271015,-0.269082
Large stocks (R_t_mom_B),-0.040855,3.641729,-0.011219,0.015334,0.090916


### Task 7

Re-do Table 3 for the decile portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- "Portfolios Formed on Book-to-Market"

In [28]:
por_book_market = pd.read_csv("Portfolios_Formed_on_BE-ME.CSV", index_col=0, header=14, nrows=1155)
por_book_market.index = pd.to_datetime(por_book_market.index.astype("str"), format="%Y%m").to_period('M').to_timestamp('M')
por_book_market = por_book_market.loc["1994":"2022"].iloc[:, 9:]

momentum_constr = pd.DataFrame().rename_axis("1994-2022", axis=1)
r_mom_d = pd.DataFrame()
i = por_book_market.columns
r_mom_d["R_mom_D1"] = por_book_market[i[-1]] - por_book_market[i[0]]
r_mom_d["R_mom_D3"] = ((por_book_market[i[-3]] + por_book_market[i[-2]] + por_book_market[i[-1]]) / 3 - (por_book_market[i[0]] + por_book_market[i[1]] + por_book_market[i[2]]) / 3)
r_mom_d["R_mom_D5"] = ((por_book_market[i[-5]] + por_book_market[i[-4]] + por_book_market[i[-3]] + por_book_market[i[-2]] + por_book_market[i[-1]]) / 5 -
                       (por_book_market[i[0]] + por_book_market[i[1]] + por_book_market[i[2]] + por_book_market[i[3]] + por_book_market[i[4]]) / 5)

momentum_constr["mean"], momentum_constr["vol"], momentum_constr["Sharpe"], momentum_constr["skewness"] = r_mom_d.mean(), r_mom_d.std(), r_mom_d.mean() / r_mom_d.std(), r_mom_d.skew()

for df in r_mom_d.columns:
    momentum_constr.loc[df, "corr_to_Rm"] = r_mom_d[df].corr(factors["MKT"])
    momentum_constr.loc[df, "corr_to_Rv"] = r_mom_d[df].corr(factors["HML"])
momentum_constr

1994-2022,mean,vol,Sharpe,skewness,corr_to_Rm,corr_to_Rv
R_mom_D1,0.098754,5.106326,0.019339,-0.152904,0.178362,0.758798
R_mom_D3,0.048029,3.350795,0.014334,-0.289519,0.174831,0.819238
R_mom_D5,0.001467,2.373432,0.000618,-0.039878,0.158168,0.831363
